In [40]:
import os
import openai
import pandas as pd
from dotenv import dotenv_values
from langchain.document_loaders import ConfluenceLoader
config = dotenv_values(".env")

openai.organization = config["ORG_ID"]
openai.api_key = config["OPENAI_API_KEY"]

config = {"persist_directory":"./chroma_db/",
          "confluence_url":"https://navihq.atlassian.net/wiki/",
          "username":"rohit.mishra@navi.com",
          "api_key":"ATATT3xFfGF0nUkk24Y9RVgJlJrKy-ScI8aDY_VCAHrr7qIKVs0GNVDRoQ0GQqNoFGZlts1A3ywX9fUa3uxzzWtSM1FxikhJKfDkeDhRAkXcHZqjwtQFxZVXipj-qYG82bGUETV_4qNAyo3GRl8XcpUdbSxQS5kyDUZBGt5RW6-My8_7mZzT5Es=10FDA744",
          "space_key":"DEVPRO"
          }

persist_directory = config.get("persist_directory",None)
confluence_url = config.get("confluence_url",None)
username = config.get("username",None)
api_key = config.get("api_key",None)
space_key = config.get("space_key",None)

## 1. Extract the documents
loader = ConfluenceLoader(
    url=confluence_url,
    username = username,
    api_key= api_key
)
documents = loader.load(
    space_key=space_key,
    page_ids=list("593429341"),
    limit=1
    )

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=10, encoding_name="cl100k_base")  # This the encoding for text-embedding-ada-002
texts = text_splitter.split_documents(texts)


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?